In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plot
import matplotlib
import chart_studio.plotly as py
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pycountry
import plotly.graph_objects as go

In [ ]:
def getDataFrameFromTable(tableName):
    df = pd.DataFrame()
    connection=None
    try:
        # Create an engine instance
        engine = postgresEngine
        # Connect to PostgreSQL server
        connection = engine.connect()
        df= pd.read_sql("select * from \""+tableName+"\" ", connection);
    except Exception as e:
        print('Exception in createDataFramefromTbl----->\n',e)
    finally:
        if connection is not None:
            connection.close()
    return df

In [ ]:
resultantDf = getDataFrameFromTable(resultantTableName)

In [ ]:
groupByCountryDf = resultantDf.loc[:,("population", "continent_name",'country_name','Air_Quality_Index','Carbon_Monoxide','Nitrogen_dioxide','Ozone','Fine_Particles_matter','Coarse_particulate_Matter','temperature','humidity', 'pressure')]
groupByCountryDf=groupByCountryDf.groupby(['country_name']).mean().reset_index()
groupByCountryDf


In [ ]:
groupByContinentDf = resultantDf.loc[:,("population", "continent_name",'country_name','Air_Quality_Index','Carbon_Monoxide','Nitrogen_dioxide','Ozone','Fine_Particles_matter','Coarse_particulate_Matter','temperature','humidity', 'pressure', 'wind_speed', 'wind_deg')]
groupByContinentDf=groupByContinentDf.groupby(['continent_name']).mean().reset_index()
groupByContinentDf

## Does high temperature lead to degradation in the quality of Air?

In [ ]:
graphPlot = sns.jointplot(x='temperature',y='Air_Quality_Index',color="#7772c0",data=resultantDf,kind='hex')
graphPlot.fig.suptitle("Air Quality Index vs Temperature Plot", fontsize=20)
graphPlot.fig.tight_layout()
graphPlot.fig.subplots_adjust(top=0.90)
graphPlot.fig.set_size_inches(10, 10)


## What is the most common temperature range based on observed average temperature of all cities?

In [ ]:
graphPlot = sns.distplot(resultantDf['temperature'],kde=True,bins=30, color='#9a73dd')
graphPlot.set_title("Temperature Distribution", fontsize=20)

## Ozone Distribution

In [ ]:
sns.distplot(resultantDf['Ozone'], bins=30, color='#db5b1f').set_title("Ozone Distribution", fontsize=20)

## Co-relation between Fine_Particles_matter vs Coarse_particulate_Matter vs Humidity vs Dew Point

In [ ]:
graphPlot = sns.pairplot(resultantDf[['Fine_Particles_matter', 'Coarse_particulate_Matter', 'humidity', 'dew_point']])
graphPlot.fig.suptitle("Fine_Particles_matter vs Coarse_particulate_Matter vs Humidity vs Dew Point",  fontsize=20)
graphPlot.fig.tight_layout()
graphPlot.fig.subplots_adjust(top=0.95)

## Which weather type is most common for each continent?

In [ ]:
graphPlot = sns.catplot(x="weather_title", hue="continent_name",
                data=resultantDf, kind="count");
graphPlot.fig.suptitle("Types Of Weather Across Continents", fontsize=20)
graphPlot.fig.tight_layout()
graphPlot.fig.subplots_adjust(top=0.95)
graphPlot.fig.set_size_inches(12,12)
handles = g._legend_data.values()
labels = g._legend_data.keys()
g.fig.legend(handles=handles, labels=labels, loc='upper left', ncol=3)
g.fig.subplots_adjust(top=0.92, bottom=0.08)

## What is the effect of temperature on overall weather of a continent?

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})

graphPlot = sns.violinplot(x="continent_name", y="temperature", data=resultantDf,hue='weather_title',palette='Set1')
graphPlot.set_title("Temperature Range Across Continents Labelled By Weather Types", fontsize=20)


In [ ]:

sns.set(rc={'figure.figsize':(11.7,8.27)})
graphPlot = sns.barplot(x='pressure',y='Air_Quality_Index',data=groupByCountryDf)
graphPlot.set_title("Co-relation Of Pressure & Air Quality Index", fontsize=20)

In [ ]:
data = dict(
        type = 'choropleth',
        colorscale = 'aggrnyl',
        locations = resultantDf['country_code'],
        z = resultantDf['population'],
        text = resultantDf['country_name'],
        colorbar = {'title' : 'GDP Billions US'},
      ) 

In [ ]:
def findCountry (country_name):
    try:
        return pycountry.countries.get(name=country_name).alpha_3
    except:
        return ("not founded")


In [ ]:
def parseCountryCodeInCountryDf():
    resultantDf['country_alpha_3'] = resultantDf.apply(lambda row: findCountry(row.country_name) , axis = 1);
parseCountryCodeInCountryDf()

## What is the most common temperature range based on observed average temperature of all cities?

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations = resultantDf['country_alpha_3'],
    z = resultantDf['temperature'],
    text = resultantDf['country_name'],
    colorscale = 'tealrose',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '',
    colorbar_title = 'Temperature<br> in deg celsius',
))

fig.update_layout(
    title_text='Temperature Distribution',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Temperature spread',
        showarrow = False
    )]
)

fig.show()